In [1]:
import requests
from scrapy.http import TextResponse
import re
import json

In [2]:
mainUrl = "https://www.libelle-lekker.be/zoeken/recepten?page="
receptUrlPrefix = "https://www.libelle-lekker.be"
user_agent = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/58: .0.3029.110 Chrome/58.0.3029.110 Safari/537.36'}
page = 1
receptUrlPrefix = "https://www.libelle-lekker.be"



In [3]:
def getResponse(url, user_agent):
    r = requests.get(url, headers=user_agent)
    return TextResponse(r.url, body=r.text, encoding='utf-8')

In [4]:
def title(jsonContent):
    return jsonContent['name']

In [5]:
def rating(jsonContent):
    return int(jsonContent['aggregateRating']['ratingValue'])

In [6]:
# Inhoud ingredienten
def ingredients(jsonContent):
    return jsonContent['ingredients']

In [7]:
def instructions(jsonContent):
    return jsonContent['recipeInstructions']

In [8]:
def labels(jsonContent):
    labels = []
    labels.append(jsonContent['recipeCuisine'])
    labels.append(jsonContent['recipeCategory'])
    return labels

In [11]:
data = {}
data['recipes']=[]
index = 0
page = 451
while True:
    print("PAGE " + str(page))
    mainResponse = getResponse(mainUrl + str(page), user_agent)
    receptPages = mainResponse.xpath('//div[@class="products__inner"]/a')
    if (len(receptPages) == 0):
        break
    #alle recepten op een pagina
    for receptUrl in receptPages:
        index = index + 1
        print("RECEPT " + str(index))
        url = receptUrl.xpath("@href").extract()[0]
        print(url)
        response = getResponse(receptUrlPrefix + url, user_agent)
        #Extract json content
        jsonText = response.xpath('//main [@class="wrapper__content"]/script/text()').extract()[0]
        #sometimes the json is 'missing' a comma between list items (incorrect json)
        #probably used as indication that these items belong to a different subsection
        jsonText = re.sub('"[\t\r\n ]+"','","',jsonText)
        #newline character
        jsonText = re.sub('[\r\n]+',' ',jsonText)
        #list item without content, not even empty string
        jsonText = re.sub(',[ ]*[\]]',']',jsonText)
        #change backslash character
        jsonText = re.sub('\\\\','/',jsonText)
        
        #print(jsonText)
        jsonContent = json.loads(jsonText)

        data['recipes'].append({
            'url':receptUrl.xpath("@href").extract()[0],
            'name':title(jsonContent),
            'rating':rating(jsonContent),
            'maxRating':5,
            'ingredients':ingredients(jsonContent),
            'instructions':instructions(jsonContent),
            'labels':labels(jsonContent)
        })
    if page%50 == 0:
        print('EXPORTING TO ' + 'data' + str(page) + '.txt')
        with open('data' + str(page) + '.txt', 'w', encoding='utf8') as outfile:  
            json.dump(data, outfile, ensure_ascii=False)
        data['recipes']=[]
    
    page = page+ 1

with open('data.txt', 'w', encoding='utf8') as outfile:  
    json.dump(data, outfile, ensure_ascii=False)

PAGE 451
RECEPT 1
/bekijk-recept/1110/spaghetti-met-amandelpesto
RECEPT 2
/bekijk-recept/1135/volkorenbrood
RECEPT 3
/bekijk-recept/1124/gevulde-tomaten
RECEPT 4
/bekijk-recept/1111/warme-aardappelsalade-met-zalm
RECEPT 5
/bekijk-recept/1119/koekjestoren-met-chocoladepudding
RECEPT 6
/bekijk-recept/1130/gedroogde-vruchten-met-wijn
RECEPT 7
/bekijk-recept/1137/aardappelbrood
RECEPT 8
/bekijk-recept/1118/chocolademousse-met-peer
RECEPT 9
/bekijk-recept/1134/meergranenbrood
RECEPT 10
/bekijk-recept/1128/spaanse-omelet-met-uien-en-bonen
RECEPT 11
/bekijk-recept/1136/witbrood
RECEPT 12
/bekijk-recept/1131/scampis-met-honing-citroenvinaigrette
RECEPT 13
/bekijk-recept/1139/knolselderroomsoep
RECEPT 14
/bekijk-recept/1117/rijstpap-met-chocolade-en-amandelen
RECEPT 15
/bekijk-recept/1125/kabeljauw-met-kruidenkorst
RECEPT 16
/bekijk-recept/1157/grietrolletjes-met-asperges
RECEPT 17
/bekijk-recept/1133/roerbakkalkoentje-met-groenten
RECEPT 18
/bekijk-recept/1129/aardappelomelet-met-spinazie-en-f

KeyboardInterrupt: 